<a href="https://colab.research.google.com/github/GuillermoSainz07/Manim-Animaciones/blob/main/Newton_Raphson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

In [ ]:
from manim import *
import numpy as np
import pandas as pd
import sympy as sp
import math as mt
from sympy.plotting import plot
import matplotlib.pyplot as plt

Manim Community v0.17.3

Crearemos una funcion que nos ayude a encontrar las raices con el metodo de newton raphson y posteriormente utilizaremos manim para una mejor visualizacion y entendiemiento del metodo

In [ ]:
raices = []
valuado = []
def Decorador(func):
  def Newton(x0,n):
    x = sp.Symbol('x')
    funcion = func(x)
    x_prima = funcion.diff(x)
    for i in range(n):
      x1 = x0 - funcion.subs(x,x0)/x_prima.subs(x,x0)
      x0 = x1
      raices.append(x1)
      valuado.append(funcion.subs(x,x0))
    print('Este numero es el valor aproximado a la raiz de la funcion: ')
    return x1
  return Newton



@Decorador # Aqui puedes agregar la funcion que desees
def calcular(x):
  fx = 0.001*(x**3) -0.5
  #p1 = plot(fx, show=False)
  #p1.show()
  return fx

calcular(25, 5)

Este numero es el valor aproximado a la raiz de la funcion: 


7.93946825279521

In [ ]:
%%manim -ql -v WARNING Newton_Rapson

class Newton_Rapson(Scene):
    def construct(self):
        axes = Axes(x_range=[-3,35],
                   y_range=[-3,20]).scale(0.8).shift(LEFT*1.1)

        function = axes.plot(lambda x: 0.001*(x**3) -0.5,
                             color=YELLOW,
                            x_range=[-1,28])


        dot_0 = Dot().move_to(axes.c2p(25, function.underlying_function(25))).set_color(RED)


        secant_0 = axes.get_secant_slope_group(x=25,
                                               graph=function,
                                               secant_line_length=7.5,
                                               secant_line_color=PURPLE,
                                               dx=0.05)



        dash_0 = DashedLine(start=axes.c2p(25,0),
                        end=axes.c2p(25, function.underlying_function(25)))

        text = Text('Metodo Newton-Raphson').scale(0.5).to_edge(UP, buff=0.5)

        text_1 = MathTex('x_{n+1} = x_n - \\frac{f(x_n)}{f`(x_n)}').scale(0.7).to_edge(UR, buff=0.7).shift(DOWN*0.5)

        text_ite = Text('Iteraciones: ').scale(0.4).next_to(text_1, DOWN).shift(LEFT*0.75)

        text_2 = Text('Raiz: ').scale(0.4).next_to(text_ite, DOWN).shift(LEFT)

        text_3 = Text('Error: ').scale(0.4).next_to(text_2, DOWN)


        self.play(DrawBorderThenFill(axes), Write(text), Write(text_1), Write(text_ite), Write(text_2), Write(text_3))
        self.play(Create(function))
        self.play(Create(dot_0))
        self.play(Create(dash_0))
        self.play(Create(secant_0))
        self.wait()

        z = 3.25
        b = 1
        ite_n = ValueTracker(b)
        ite = always_redraw(lambda: Integer(ite_n.get_value()).scale(0.7).set_color(YELLOW).next_to(text_ite, RIGHT))

        error_n = ValueTracker(10)
        error_num = always_redraw(lambda: DecimalNumber(error_n.get_value(), num_decimal_places=3).scale(0.7).set_color(YELLOW).next_to(text_3, RIGHT))
        raiz_n = ValueTracker(25)
        raiz_num = always_redraw(lambda: DecimalNumber(raiz_n.get_value(), num_decimal_places=2).scale(0.7).set_color(YELLOW).next_to(text_2, RIGHT))
        self.play(Create(raiz_num), Create(error_num), Create(ite))

        for raiz, val in zip(raices, valuado):

          dot_x = Dot().move_to(axes.c2p(raiz, 0)).set_color(BLUE)
          self.play(Create(dot_x))
          dash_x_to_y = DashedLine(start=axes.c2p(raiz, 0),
                                   end=axes.c2p(raiz, function.underlying_function(raiz)))
          self.play(Create(dash_x_to_y))
          dot_y = Dot().move_to(axes.c2p(raiz, function.underlying_function(raiz))).set_color(RED)
          self.play(Create(dot_y))
          secant_n = axes.get_secant_slope_group(x=raiz,
                                                 graph=function,
                                                 secant_line_length=z,
                                                 secant_line_color=PURPLE,
                                                 dx=0.05)
          z/=2
          b+=1
          self.play(ite_n.animate.set_value(b))
          self.play(raiz_n.animate.set_value(raiz))
          self.play(error_n.animate.set_value(np.float128(val)))
          self.play(Create(secant_n))
          self.wait()

        rec = SurroundingRectangle(dot_y, color=GREEN)
        rec_raiz = SurroundingRectangle(raiz_num, color=GREEN)
        rec_error = SurroundingRectangle(error_num, color=GREEN)
        self.play(Create(rec), Create(rec_raiz))
        self.wait()
